In [1]:
from milvus import default_server
from pymilvus import connections, utility

## Connects to a server:

In [2]:
default_server.start()



    __  _________ _   ____  ______
   /  |/  /  _/ /| | / / / / / __/
  / /|_/ // // /_| |/ / /_/ /\ \
 /_/  /_/___/____/___/\____/___/ {Lite}

 Welcome to use Milvus!

 Version:   v2.2.8-lite
 Process:   49802
 Started:   2023-06-08 10:48:14
 Config:    /home/nam/.milvus.io/milvus-server/2.2.8/configs/milvus.yaml
 Logs:      /home/nam/.milvus.io/milvus-server/2.2.8/logs

 Ctrl+C to exit ...


In [3]:
connections.connect(host='0.0.0.0', port=19530)

print(utility.get_server_version())

default_server.stop()

v2.2.9


In [4]:
from pymilvus import (
    connections,
    utility,
    FieldSchema,
    CollectionSchema,
    DataType,
    Collection,
)

In [5]:
connections.connect(host='0.0.0.0', port=19530)

## Creates a collection:

In [6]:
fields = [
    FieldSchema(name="pk", dtype=DataType.INT64, is_primary=True, auto_id=False),
    FieldSchema(name="random", dtype=DataType.DOUBLE),
    FieldSchema(name="embeddings", dtype=DataType.FLOAT_VECTOR, dim=8)
]
schema = CollectionSchema(fields, "hello_milvus is the simplest demo to introduce the APIs")
hello_milvus = Collection("hello_milvus", schema)

In [7]:
hello_milvus

<Collection>:
-------------
<name>: hello_milvus
<description>: hello_milvus is the simplest demo to introduce the APIs
<schema>: {'auto_id': False, 'description': 'hello_milvus is the simplest demo to introduce the APIs', 'fields': [{'name': 'pk', 'description': '', 'type': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': False}, {'name': 'random', 'description': '', 'type': <DataType.DOUBLE: 11>}, {'name': 'embeddings', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 8}}]}

## Inserts vectors in the collection:

In [8]:
import random
entities = [
    [i for i in range(3000)],  # field pk
    [float(random.randrange(-20, -10)) for _ in range(3000)],  # field random
    [[random.random() for _ in range(8)] for _ in range(3000)],  # field embeddings
]
insert_result = hello_milvus.insert(entities)
hello_milvus.flush()  

In [9]:
print(connections.list_connections())

[('default', <pymilvus.client.grpc_handler.GrpcHandler object at 0x7fb82638deb0>)]


## Builds indexes on the entities:

In [10]:
index = {
    "index_type": "IVF_FLAT",
    "metric_type": "L2",
    "params": {"nlist": 128},
}
hello_milvus.create_index("embeddings", index)

Status(code=0, message=)

## Loads the collection to memory and performs a vector similarity search:

In [11]:
hello_milvus.load()
vectors_to_search = entities[-1][-2:]
vectors_to_search

[[0.8138255139550796,
  0.8090067213893002,
  0.8766097362926442,
  0.8065852284577353,
  0.6945133952344488,
  0.21997777756590386,
  0.03389606839399806,
  0.965710172594879],
 [0.8610076224328155,
  0.3771223632018438,
  0.5570885138158789,
  0.3003421712056409,
  0.5370515069295857,
  0.8823157302111319,
  0.995880448528799,
  0.4740953338943459]]

In [12]:

search_params = {
    "metric_type": "L2",
    "params": {"nprobe": 12},
}
results = hello_milvus.search(vectors_to_search, "embeddings", search_params, limit=5, output_fields=["pk","random"])

In [13]:
for i, result in enumerate(results):
    print("\nSearch result for {}th vector: ".format(i))
    for j, res in enumerate(result):
        print("Top {}: {}".format(j, res))


Search result for 0th vector: 
Top 0: id: 2998, distance: 0.0, entity: {'random': -12.0, 'pk': 2998}
Top 1: id: 1584, distance: 0.0347994863986969, entity: {'random': -13.0, 'pk': 1584}
Top 2: id: 727, distance: 0.0840626209974289, entity: {'random': -16.0, 'pk': 727}
Top 3: id: 2705, distance: 0.15021929144859314, entity: {'random': -17.0, 'pk': 2705}
Top 4: id: 1573, distance: 0.1505608856678009, entity: {'random': -12.0, 'pk': 1573}

Search result for 1th vector: 
Top 0: id: 2999, distance: 0.0, entity: {'random': -13.0, 'pk': 2999}
Top 1: id: 284, distance: 0.12008361518383026, entity: {'random': -16.0, 'pk': 284}
Top 2: id: 2079, distance: 0.1205524206161499, entity: {'random': -13.0, 'pk': 2079}
Top 3: id: 1125, distance: 0.12374809384346008, entity: {'random': -15.0, 'pk': 1125}
Top 4: id: 2401, distance: 0.1386527717113495, entity: {'random': -13.0, 'pk': 2401}


## Performs a vector query:

In [14]:
result = hello_milvus.query(expr="random > -12", output_fields=["random", "embeddings"])
result

[{'random': -11.0,
  'embeddings': [0.85178286,
   0.9802736,
   0.7072149,
   0.51051134,
   0.28189304,
   0.50286394,
   0.7754523,
   0.5618342],
  'pk': 6},
 {'random': -11.0,
  'embeddings': [0.012056025,
   0.91466695,
   0.9251583,
   0.45067492,
   0.63728505,
   0.58820057,
   0.088610955,
   0.52077794],
  'pk': 12},
 {'random': -11.0,
  'embeddings': [0.78719217,
   0.7333356,
   0.58954716,
   0.0033241862,
   0.5361536,
   0.009867032,
   0.041557692,
   0.4645213],
  'pk': 20},
 {'random': -11.0,
  'embeddings': [0.59757024,
   0.18332824,
   0.64325076,
   0.62063533,
   0.11719743,
   0.1733666,
   0.75012356,
   0.26791516],
  'pk': 22},
 {'random': -11.0,
  'embeddings': [0.29598328,
   0.8519248,
   0.42453757,
   0.9747178,
   0.49313888,
   0.6943806,
   0.7258842,
   0.7021146],
  'pk': 25},
 {'random': -11.0,
  'embeddings': [0.9203923,
   0.45629996,
   0.78443545,
   0.3107109,
   0.86026245,
   0.7685671,
   0.021069242,
   0.044061273],
  'pk': 32},
 {'rando

## Performs a hybrid search:

In [15]:
results = hello_milvus.search(vectors_to_search, "embeddings", search_params, 
                              limit=3, expr="random > -12", output_fields=["random"])

for i, result in enumerate(results):
    print("\nSearch result for {}th vector: ".format(i))
    for j, res in enumerate(result):
        print("Top {}: {}".format(j, res))


Search result for 0th vector: 
Top 0: id: 2107, distance: 0.1837185025215149, entity: {'random': -11.0}
Top 1: id: 2469, distance: 0.26508504152297974, entity: {'random': -11.0}
Top 2: id: 2714, distance: 0.3231860399246216, entity: {'random': -11.0}

Search result for 1th vector: 
Top 0: id: 535, distance: 0.18041419982910156, entity: {'random': -11.0}
Top 1: id: 142, distance: 0.19035759568214417, entity: {'random': -11.0}
Top 2: id: 1874, distance: 0.25826799869537354, entity: {'random': -11.0}


## Drops the collection:


In [16]:
utility.drop_collection("hello_milvus")